In [1]:
import os
import pandapower as pp
import polars as pl
from polars import col as c

from general_function import dict_to_duckdb
from pipelines.dig_a_plan_model_test import DigAPlanTest
from pipelines.dig_a_plan_complete_master import DigAPlan

from data_display.grid_plotting import plot_grid_from_pandapower
from data_display.output_processing import compare_dig_a_plan_with_pandapower
from data_connector import change_schema_to_dig_a_plan_schema, duckdb_to_changes_schema
from twindigrid_changes.schema import ChangesSchema
from networkx_function import generate_nx_edge, get_connected_edges_data, generate_bfs_tree_with_edge_data, get_all_edge_data
import networkx as nx

from pyomo_utility import extract_optimization_results

import matplotlib.pyplot as plt

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"
# os.environ['MOSEKLM_LICENSE_FILE'] = os.environ["HOME"] + "/mosek/mosek.lic"


In [69]:
from matplotlib.pylab import normal


change_schema: ChangesSchema = duckdb_to_changes_schema(file_path=".cache/input_data/boisy_grid.db")
# 2) build the *original* schema once

base_grid_data = change_schema_to_dig_a_plan_schema(change_schema = change_schema, s_base=1e6)

nx_graph = nx.Graph()
_ = base_grid_data["edge_data"].with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_graph)
)

max_n_trafo = 1
edge_id  = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)["edge_id"].to_list()
node_id = get_connected_edges_data(nx_graph = nx_graph).filter(c("graph_id")!= 0)\
    .unpivot(on=["u_of_edge", "v_of_edge"])["value"].to_list()

base_grid_data["edge_data"] = base_grid_data["edge_data"].filter(~c("edge_id").is_in(edge_id))
base_grid_data["node_data"] = base_grid_data["node_data"].filter(~c("node_id").is_in(node_id + [252]))

base_grid_data["edge_data"] = base_grid_data["edge_data"].with_columns(
        pl.when(c(col) < 1e-3).then(pl.lit(0)).otherwise(c(col)).alias(col) for col in ["b_pu", "r_pu", "x_pu"]
    ).with_columns(
        pl.lit(1.0).alias("n_transfo"),
        c("normal_open").fill_null(False).alias("normal_open"),
    )
    
base_grid_data["node_data"] = base_grid_data["node_data"].with_columns(
    # (c("p_node_pu")*5e-1).alias("p_node_pu"),
    (c("p_node_pu")*1e-1).alias("q_node_pu")
    )


Read and validate tables from boisy_grid.db file:   0%|                                                                        | 0/28 [00:00<?, ?it/s]

Read and validate tables from boisy_grid.db file: 100%|██████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 271.27it/s]


In [77]:
edge_id = base_grid_data["edge_data"]\
    .unpivot(on=["u_of_edge", "v_of_edge"], index=["edge_id", "type"])\
    .filter(c("value").is_unique())\
    .filter(c("type")== "switch")["edge_id"].to_list()
    
base_grid_data["edge_data"] = base_grid_data["edge_data"].with_columns(
    pl.when(c("edge_id").is_in(edge_id)).then(pl.lit(False)).otherwise(c("normal_open")).alias("normal_open")
)

base_grid_data["edge_data"] = base_grid_data["edge_data"].filter(~c("normal_open"))

In [78]:
dict_to_duckdb(base_grid_data, file_path=".cache/input_data/start_boisy_grid.db")

Save dictionary into duckdb file: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 17.42it/s]


In [12]:
nx_graph = nx.Graph()
_ = base_grid_data["edge_data"].filter(~c("normal_open")).select(
    pl.struct("edge_id", "u_of_edge", "v_of_edge").pipe(generate_nx_edge, nx_graph=nx_graph)
)
print(nx.is_connected(nx_graph))
print(nx.is_tree(nx_graph))

True
True


In [3]:
dig_a_plan: DigAPlan = DigAPlan(
    verbose= False, big_m = 1e6, penalty_cost = 1e1, current_factor= 1, voltage_factor = 1, power_factor = 1,
    infeasibility_factor = 1
)

dig_a_plan.add_grid_data(**base_grid_data)

In [4]:
dig_a_plan.solve_models_pipeline(max_iters = 200)

Master obj: -1.0E+08, Slave obj: 4.4E+01 and Gap: INF:   2%|▏         | 3/200 [00:07<08:03,  2.45s/it]


ValueError: Cannot load a SolverResults object with bad status: error

In [5]:
nx_graph = nx.Graph()
open_switches_list = extract_optimization_results(dig_a_plan.master_model_instance, "delta").filter(c("delta") == 0.0)["S"].to_list()
_ = base_grid_data["edge_data"].filter(~c("edge_id").is_in(open_switches_list)).select(
    pl.struct("edge_id", "u_of_edge", "v_of_edge").pipe(generate_nx_edge, nx_graph=nx_graph)
)
print(nx.is_connected(nx_graph))
print(nx.is_tree(nx_graph))

True
True


In [6]:
len(open_switches_list)

49

In [ ]:
base_grid_data["edge_data"].filter(c("normal_open"))

edge_id,u_of_edge,v_of_edge,eq_fk,r_pu,x_pu,b_pu,i_max_pu,type,i_base,diff,uuid,rated_s,diff_right,uuid_right,timestamp,heartbeat,source_fk,r,r0,x,x0,b,b0,g,g0,side,vn_hv,vn_lv,v_base_hv,v_base_lv,n_transfo,z_base,normal_open,command,v_base
u32,i64,i64,str,f64,f64,f64,f64,str,f64,str,str,f64,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",str,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,i32,i32,i32,f64,f64,bool,str,i32
778,61,60,"""d9255bb0-767a-5eec-81ff-0a3df1…",null,null,null,null,"""switch""",50.204371,"""+""","""d9255bb0-767a-5eec-81ff-0a3df1…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",11500
786,92,86,"""922efaff-4656-579c-8b53-e525e2…",null,null,null,null,"""switch""",50.204371,"""+""","""922efaff-4656-579c-8b53-e525e2…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",11500
787,92,91,"""ed143c3e-82cc-5976-aa7b-e81e89…",null,null,null,null,"""switch""",50.204371,"""+""","""ed143c3e-82cc-5976-aa7b-e81e89…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",11500
790,107,104,"""0e05522f-cbe2-5b34-ae55-9a9204…",null,null,null,null,"""switch""",50.204371,"""+""","""0e05522f-cbe2-5b34-ae55-9a9204…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",11500
792,113,112,"""9bacd1d0-7b6f-591e-b3c3-369f5b…",null,null,null,null,"""switch""",50.204371,"""+""","""9bacd1d0-7b6f-591e-b3c3-369f5b…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",11500
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1038,329,252,"""35eb3bc4-8b29-526d-a962-d81238…",null,null,null,null,"""switch""",1443.375673,"""+""","""35eb3bc4-8b29-526d-a962-d81238…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",400
1040,200,271,"""b8cbc648-fdf5-5352-9339-60ae9e…",null,null,null,null,"""switch""",1443.375673,"""+""","""b8cbc648-fdf5-5352-9339-60ae9e…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",400
1041,280,185,"""c7764c22-c0dc-558b-89f4-6cc58b…",null,null,null,null,"""switch""",1443.375673,"""+""","""c7764c22-c0dc-558b-89f4-6cc58b…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,null,true,"""unknown""",400


In [ ]:
base_grid_data["edge_data"] = base_grid_data["edge_data"].filter(~c("edge_id").is_in(open_switches_list))

In [ ]:
dict_to_duckdb(base_grid_data, file_path=".cache/input_data/new_boisy_grid.db")

Save dictionary into duckdb file: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 20.12it/s]


In [ ]:


dig_a_plan_test: DigAPlanTest = DigAPlanTest(
    verbose= False, big_m = 1e6, penalty_cost = 1e1, current_factor= 1, voltage_factor = 1, power_factor = 1,
)
dig_a_plan_test.add_grid_data(**base_grid_data)



dig_a_plan_test.test_one_switch_combinations(open_switches_list=open_switches_list)

shape: (281, 2)
┌──────┬───────┐
│ S    ┆ delta │
│ ---  ┆ ---   │
│ i64  ┆ i64   │
╞══════╪═══════╡
│ 771  ┆ 1     │
│ 772  ┆ 0     │
│ 773  ┆ 1     │
│ 774  ┆ 1     │
│ 776  ┆ 1     │
│ …    ┆ …     │
│ 1054 ┆ 1     │
│ 1055 ┆ 1     │
│ 1056 ┆ 1     │
│ 1057 ┆ 1     │
│ 1058 ┆ 1     │
└──────┴───────┘
2025-06-30 08:32:41 B01BPC15 gurobipy[15905] INFO Read LP format model from file /tmp/tmp4cxhs2qt.pyomo.lp
2025-06-30 08:32:41 B01BPC15 gurobipy[15905] INFO Reading time = 0.01 seconds
2025-06-30 08:32:41 B01BPC15 gurobipy[15905] INFO x1: 7267 rows, 4209 columns, 16830 nonzeros
2025-06-30 08:32:41 B01BPC15 gurobipy[15905] INFO Set parameter IntegralityFocus to value 1
2025-06-30 08:32:41 B01BPC15 gurobipy[15905] INFO Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")
2025-06-30 08:32:41 B01BPC15 gurobipy[15905] INFO 
2025-06-30 08:32:41 B01BPC15 gurobipy[15905] INFO CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
2025-06

2025-06-30 08:32:43 B01BPC15 pipelines.dig_a_plan_model_test[15905] ERROR Slave model failed to solve: Cannot load a SolverResults object with bad status: error
